# Reading data into Astropy Tables

## Objectives

 - Read ASCII files with a defined format
 - Learn basic operations with `astropy.tables`
 - Ingest header information
 - VOTables

## Reading data

Our first task with Python was to read a `.csv` file using `np.loadtxt()`.
That function has some arguments to define the dlimiter of the columns, skip rows, read commented lines, convert values while reading, etc.

However, the result is an array, without the information of the metadata that file may have included (name, units, ...).

AstroPy offers an ascii reader that improves many of these steps and provides templates to read common ascii files in astronomy.


In [ ]:
from astropy.io import ascii

In [ ]:
# Read a sample file: sources.dat
data = ascii.read("sources.dat")
data

Automatically, `ascii.read()` has identified the header and the format of each column. The result is a `Table` object, and that brings some additional properties.

In [ ]:
# Show the info of the data read
data.info

In [ ]:
# Get the name of the columns
data.colnames

In [ ]:
# Get just the values of a particular column
data['obsid']

In [ ]:
# get the first element
data['obsid', 'redshift'][0]

Astropy [can read a variety of formats](http://astropy.readthedocs.org/en/stable/io/ascii/index.html#supported-formats) easily.
The following example uses a quite 

In [ ]:
# Read the data from the source
table = ascii.read("ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/snrs.dat",
                   readme="ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/ReadMe")

In [ ]:
# See the stats of the table
table.info('stats')

In [ ]:
# If we want to see the first 10 entries
table[0:10]

In [ ]:
# the units are also stored, we can extract them too
table['MajDiam'].quantity.to('rad')[0:3]

In [ ]:
# Adding values of different columns
(table['RAh'] + table['RAm'] + table['RAs'])[0:3]


In [ ]:
# adding values of different columns but being aware of column units
(table['RAh'].quantity + table['RAm'].quantity + table['RAs'].quantity)[0:3]

In [ ]:
# Create a new column in the table
table['RA'] = table['RAh'].quantity + table['RAm'].quantity + table['RAs'].quantity

In [ ]:
# Show table's new column 
table['RA'][0:3]

In [ ]:
# add a description to the new column
table['RA'].description = table['RAh'].description

In [ ]:
# Now it does show the values
table['RA'][0:3]


In [ ]:
# Using numpy to calculate the sin of the RA
import numpy as np
np.sin(table['RA'].quantity)

In [ ]:
# Let's change the units...
import astropy.units as u
table['RA'].unit = u.hourangle

In [ ]:
# does the sin now works?
np.sin(table['RA'].quantity)

## Properties when reading

the reading of the table has many properties, let's imagine the following easy example:

In [ ]:
weather_data = """
# Country = Finland
# City = Helsinki
# Longitud = 24.9375
# Latitud = 60.170833
# Week = 32
# Year = 2015
day, precip, type
Mon,1.5,rain
Tues,,
Wed,1.1,snow
Thur,2.3,rain
Fri,0.2,
Sat,1.1,snow
Sun,5.4,snow
"""

In [ ]:
# Read the table
weather = ascii.read(weather_data)

In [ ]:
# Blank values are interpreted by default as bad/missing values
weather.info('stats')

In [ ]:
# Let's define missing values for the columns we want:
weather['type'].fill_value = 'N/A'
weather['precip'].fill_value = -999

In [ ]:
# Use filled to show the value filled.
weather.filled()

In [ ]:
# We can see the meta as a dictionary, but not as key, value pairs
weather.meta

In [ ]:
# To get it the header as a table
header = ascii.read(weather.meta['comments'], delimiter='=',
                    format='no_header', names=['key', 'val'])
print(header)

When the values are not empty, then the keyword `fill_values` on `read` has [to be used](http://astropy.readthedocs.org/en/stable/io/ascii/read.html#bad-or-missing-values).


## Reading VOTables

VOTables are an special type of tables which should be self-consistent and can be tied to a particular scheme.
This means the file will contain where the data comes from (and which query produced it) and the properties for each field, making it easier to ingest by a machine.

In [ ]:
from astropy.io.votable import parse_single_table

In [ ]:
# Read the example table from HELIO (hfc_ar.xml)
table = parse_single_table("hfc_ar.xml")

In [ ]:
# See the fields of the table
table.fields

In [ ]:
# extract one  (NOAA_NUMBER) or all of the columns
NOAA = table.array['NOAA_NUMBER']

In [ ]:
# Show the data
NOAA.data

In [ ]:
# See the mask
NOAA.mask

In [ ]:
# Shee the whole array.
NOAA

In [ ]:
# Convert the table to an astropy table
asttable = table.to_table()

In [ ]:
# See the table
asttable

In [ ]:
# Different results because quantities are not 
print(np.sin(asttable['FEAT_HG_LAT_DEG'][0:5]))
print(np.sin(asttable['FEAT_HG_LAT_DEG'][0:5].quantity))


In [ ]:
# And it can also be converted to other units
print(asttable[0:5]['FEAT_AREA_DEG2'].quantity.to('arcmin2'))